In [ ]:
# set this to point to the parameter file used
const PARAMETER_FILE = "model.parameters.json"
# set this to point to the .results file containing the simulations
const RESULTS_PREFIX = "experiments/SR3.0/Powder"#.results, .high.results etc
const WEATHER_FILE   = "data/TGA.daily.df.csv"
const LOW_STOCKING_RESULTS = "experiments/SR2.7/Powder.results"

In [ ]:
using JSON, SDDP,DataFrames#, StatPlots
data = JSON.parsefile(PARAMETER_FILE)
results = SDDP.load(string(RESULTS_PREFIX, ".results"));
low = SDDP.load(string(RESULTS_PREFIX, ".low.results"));
high = SDDP.load(string(RESULTS_PREFIX, ".high.results"));
low_stocking = SDDP.load(LOW_STOCKING_RESULTS);

In [ ]:
using Plots, StatPlots
const mm = Plots.mm
const pt = Plots.pt
fntsm = Plots.font("times", 10.0pt)
fntlg = Plots.font("times", 12.0pt)
default(titlefont=fntlg, guidefont=fntlg, tickfont=fntsm, legendfont=fntsm,left_margin=10mm,bottom_margin=7.5mm)
default(size=(800,600),top_margin=0mm, right_margin=0mm)
gr()

In [ ]:
# Plot Figure 1 in the paper
plot(size=(1000,600), left_margin=5mm, bottom_margin=10mm)
milking_requirements = data["energy_for_pregnancy"] + data["energy_for_bcs_milking"] + data["energy_for_maintenance"]
dry_requirements = data["energy_for_pregnancy"] + data["energy_for_bcs_dry"] + data["energy_for_maintenance"]
plot!(milking_requirements, label="Total: Lactating Cow", linewidth=3, color="#00467F")
plot!(dry_requirements, label="Total: Dry Cow", linewidth=3, color="#00467F", linestyle=:dot)
plot!(data["energy_for_bcs_milking"],label="Change in BCS: Lactating Cow", linewidth=3, color="#e65100")
plot!(data["energy_for_bcs_dry"],label="Change in BCS: Dry Cow", linewidth=3, color="#e65100", linestyle=:dot)
hline!([data["energy_for_maintenance"]],label="Maintenance", linewidth=3, color="#009AC7", linestyle=:solid)
plot!(data["energy_for_pregnancy"], label="Pregnancy", linewidth=3, color="#009AC7", linestyle=:dash)
plot!(ylims=(-100,900), xlabel="Weeks since calving\n", ylabel="Energy Requirement (MJ/week)", legend=:topleft)
savefig("energy.pdf")

In [ ]:
# Plot Figure 4
hgh = 3
plot(xlabel="Quantity of Palm Kernel Fed (kg/cow/day)\n", ylabel="Penalty (\$/cow/day)",legend=false,ylims=(0,1.25))
xbx = [0.0, 2.0, 2.0, 0.0]
ybx = [0, 0, 2hgh, 2hgh]
plot!(Plots.Shape(xbx, ybx), fillalpha=0.5, w=0, c="green", alpha=0)
plot!(Plots.Shape(2+xbx, ybx), fillalpha=0.5, w=0, c="orange", alpha=0)
plot!(Plots.Shape(4+xbx, ybx), fillalpha=0.5, w=0, c="red", alpha=0)
plot!([0, 3, 4, 5, 6], [0, 0, 0.25, 0.75,1.75], w=5, c="#00467F")
plot!(size=(4 * 175,2 * 175), left_margin=5mm, bottom_margin=10mm)
annotate!([(1,0.4,text("Fat Evaluation Index\nGrade A")), (3,0.4,text("Fat Evaluation Index\nGrade B")), (5,0.4,text("Fat Evaluation Index\nGrade C"))])
savefig("fei_penalty.pdf")

In [ ]:
# Plot Figure 5
using CSV
df = CSV.read(WEATHER_FILE)
q = [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
rainfall = unstack(by(df, :week) do io
   DataFrame(
        rainfall = quantile(io[:rainfall], q),
        quantile = q
        ) 
        end, :quantile, :rainfall)
rainfall = rainfall[vcat(35:52, 1:34),:]
rename!(rainfall, [f => t for (f,t) in zip(names(rainfall), [:week, :a, :b, :c, :d, :e, :f, :g])])
plot(rainfall[:d], w=3, c="#00467F")
plot!(rainfall[:a], fill=(rainfall[:g], "#00467F"), fillalpha=0.25, alpha=0.0)
plot!(rainfall[:b], fill=(rainfall[:f], "#00467F"), fillalpha=0.25, alpha=0.0)
plot!(rainfall[:c], fill=(rainfall[:e], "#00467F"), fillalpha=0.25, alpha=0.0)

xticks = collect(1:8.66:52)
xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]

rainfall_plot = plot!(
    legend=false,
    title="(a)", xlims=(1,52), xticks=(xticks, xticklabels),
    ylabel="Rainfall\n(mm/week)",
)
evapotranspiration = unstack(by(df, :week) do io
   DataFrame(
        evapotranspiration = quantile(io[:evapotranspiration], q),
        quantile = q
        ) 
        end, :quantile, :evapotranspiration)
evapotranspiration = evapotranspiration[vcat(35:52, 1:34),:]
rename!(evapotranspiration, [f => t for (f,t) in zip(names(evapotranspiration), [:week, :a, :b, :c, :d, :e, :f, :g])])
plot(evapotranspiration[:d], w=3, c="#00467F")
plot!(evapotranspiration[:a], fill=(evapotranspiration[:g], "#00467F"), fillalpha=0.25, alpha=0.0)
plot!(evapotranspiration[:b], fill=(evapotranspiration[:f], "#00467F"), fillalpha=0.25, alpha=0.0)
plot!(evapotranspiration[:c], fill=(evapotranspiration[:e], "#00467F"), fillalpha=0.25, alpha=0.0)
evapotranspiration_plot = plot!(
    legend=false, title="(b)",
    xticks=(xticks, xticklabels),
    ylabel="Evapotranspiration\n(mm/week)"
)
plot(rainfall_plot, evapotranspiration_plot, layout=(1,2), size=(1000,375), left_margin=8mm, bottom_margin=5mm)
savefig("weather.pdf")

In [ ]:
# Plot Figures 5 and 6
function getresults(results, key, scalefactor=1.0)
    Q = [0.0, 0.05, 0.5, 0.95, 1.0]
    g2 = hcat([scalefactor*r[key] for r in results]...)
    g3 = hcat([quantile(g2[g,:], Q) for g in 1:size(g2, 1)]...)'
    df = DataFrame(week=1:size(g3, 1), x10=g3[:,1], x25=g3[:,2], x50=g3[:,3], x75=g3[:,4], x90=g3[:,5])
    for key in names(df)
        df[key] = convert(Vector{Union{eltype(df[key]), Missings.Missing}}, df[key])
    end
    df
end
function plotpowder(islow, low, high, key, ylims, ylabel,title,scalefactor=1.0)
    wks = 24:52
    blue = getresults(low, key, scalefactor)[wks, :]
    orange = getresults(high, key, scalefactor)[wks, :]

    gray = getresults(vcat(low, high), key, scalefactor)[1:24, :]

    plot()
    plot!(gray[:week], gray[:x10], fill=(gray[:x90], "gray"), fillalpha=0.25, alpha=0.0)
    plot!(gray[:week], gray[:x25], fill=(gray[:x75], "gray"), fillalpha=0.25, alpha=0.0)
    plot!(gray[:week], gray[:x50], c="gray", w=2, linestyle=:dot)
    
    if islow
        plot!(blue[:week], blue[:x10], fill=(blue[:x90], "#00467F"), c="#00467F", fillalpha=0.3, alpha=0)
        plot!(blue[:week], blue[:x25], fill=(blue[:x75], "#00467F"), c="#00467F", fillalpha=0.3, alpha=0)
        plot!(blue[:week], blue[:x50], c="#00467F", w=2, linestyle=:dot)
    else
        plot!(orange[:week], orange[:x10], fill=(orange[:x90], "#e65100"), c="#e65100", fillalpha=0.3, alpha=0)
        plot!(orange[:week], orange[:x25], fill=(orange[:x75], "#e65100"), c="#e65100", fillalpha=0.3, alpha=0)
        plot!(orange[:week], orange[:x50], c="#e65100", w=2, linestyle=:dot)
    end
    plot!(1:24, scalefactor*high[10][key][1:24], color="gray", linewidth=3)
    if islow
        plot!(wks, scalefactor*low[10][key][wks], color="#00467F", linewidth=3)
    else
        plot!(wks, scalefactor*high[10][key][wks], color="#e65100", linewidth=3)
    end
    xticks = collect(1:8.66:52)
    xticklabels = ["Aug", "Oct", "Dec", "Feb", "Apr", "Jun"]
    plot!(legend=false, ylims=ylims, ylabel=ylabel,title=title, xlims=(1,52), xticks=(xticks, xticklabels), xlabel="")
end

function plotall(islow, low, high)
    plot(
        plotpowder(islow, low, high, :C, (0, 3), "Cows Milking\n(cows/ha)", "(a)"),
        plotpowder(islow, low, high, :P, (1.25,3.25), "Pasture Cover\n(t/ha)","(b)", 0.001),
        plotpowder(islow, low, high, :W, (0,150), "Soil Moisture\n(mm)","(c)"),
        plotpowder(islow, low, high, :gr, (0,70), "Pasture Growth\n(kg/day)", "(d)"),
        plotpowder(islow, low, high, :b, (0,6), "Palm Kernel Fed\n(kg/cow/day)", "(e)", 1 / 3 / 7),
        plotpowder(islow, low, high, :milk, (0,2.5), "Milk Production\n(kg/cow/day)", "(f)", 1 / 3 / 7),
        layout=(2,3), size=(1500,600)
    )
end
plotall(false, low, high)
savefig("farm_high.pdf")
plotall(true, low, high)
savefig("farm_low.pdf")

In [ ]:
# Plot Figure 7
function plotdensity(df, label, linestyle, color="#00467F")
    fixedcost = data["fixed_cost"]
    density!([r[:objective]-fixedcost for r in df], label=label, color=color, linewidth=3, linestyle=linestyle)
    vline!([mean([r[:objective]-fixedcost for r in df])], label="", color=color, linewidth=3)
end
plot(size=(4 * 150,3 * 150),
    xlabel="Operating Profit (\$/ha)\n",
    xlims=(-4000, 8000), 
    ylabel="Smoothed Density", yticks=false)
plotdensity(low_stocking, "2.7 cows/ha", :solid)
plotdensity(results, "3.0 cows/ha", :dot, "#e65100")
plot!(ylims=(0, 0.00035), legend=:topleft, bottom_margin=7.0Plots.mm, right_margin=5Plots.mm)
savefig("density.pdf")

In [ ]:
fixedcost = data["fixed_cost"]
@show mean([r[:objective]-fixedcost for r in results])
@show mean([r[:objective]-fixedcost for r in low_stocking])

In [ ]:
# Plot Figure 8
obj = Float64[]
SR = [1.8, 2.1, 2.4, 2.7, 3.0]
for sr in SR
    sr_results = SDDP.load(string("experiments/SR", sr, "/POWDER.results"))
    fixedcost = data["fixed_cost"]
    push!(obj, mean([r[:objective]-fixedcost for r in sr_results]))
end
plot(size=(4 * 150,3 * 150),
    xlabel="Stocking Rate (cows/Ha)\n",
#     xlims=(-4000, 8000), 
    ylabel="Operating Profit (\$/Ha)")
plot!(SR, obj, linewidth=3, color="#00467F")
plot!(legend=false, bottom_margin=7.0Plots.mm, right_margin=5Plots.mm, ylims=(2000, 3000), xlims=(1.8, 3.0), xticks=(SR))
savefig("stockingrate.pdf")

In [ ]:
# Plot Figure 8
SR = [1.8, 2.1, 2.4, 2.7, 3.0]
function plotquantile(SR, q)
    obj = Float64[]
    for sr in SR
        sr_results = SDDP.load(string("experiments/SR", sr, "/POWDER.results"))
        i = 0.0
        for r in sr_results
            if sum(r[:mlk][1:44]) >= q * sum(data["max_milk_energy"][1:44]) * sr - 1
                i+=1.0
            end
        end
        push!(obj, i / length(sr_results))
    end
    plot!(SR, obj, linewidth=1)#3, color="#00467F")
end

plot(size=(4 * 150,3 * 150),
    xlabel="Stocking Rate (cows/Ha)\n",
#     xlims=(-4000, 8000), 
    ylabel="Operating Profit (\$/Ha)")
plotquantile(SR, 1.0)
plotquantile(SR, 0.9)
plotquantile(SR, 0.8)
plotquantile(SR, 0.7)
plot!(legend=false, bottom_margin=7.0Plots.mm, right_margin=5Plots.mm, ylims=(0, 1), xlims=(1.8, 3.0), xticks=(SR))
savefig("stockingrate.pdf")